# Finetuning

1. Upload an eval_config file into this directory, save it as `config.json`
2. Add a `.env` file, the contents should be:
```
export HUGGINGFACE_API_KEY=xxxx
```

where `xxxx` is replaced with the key


## Install Prerequisite Packages

In [3]:
# This is necessary for colab
!pip install python-dotenv
!pip install datasets
!pip install plotly
!pip install nbformat
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-7f6e081a/unsloth_03bfaa4f84b84b5e9e0ee297c121eb48
  Running command git clone --filter=bl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 101.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 22.6 MB/s eta 0:00:00
  Attempting uninstall: trl
    Found existing installation: trl 0.12.1
    Uninstalling trl-0.12.1:
      Successfully uninstalled trl-0.12.1


## Imports, Load `.env`

In [4]:
import os
import sys
import json
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template
import torch

from datasets import Dataset
from datasets import load_dataset

from dotenv import find_dotenv, load_dotenv

from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported
from unsloth.chat_templates import train_on_responses_only

# The file .env should contain the line (where xxxxxxx is replaced):
# export HUGGINGFACE_API_KEY=xxxxxxx
load_dotenv()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


True



## Load Config

In [5]:
FINETUNING_DATASET_NAME="CPSC532/arxiv_qa_data"
BATCH_SIZE = 1
GRADIENT_ACCUMULATION_STEPS = 4
PACKING = True

with open('config.json') as f:
    config = json.load(f)
finetune_config_template = config["template"]["finetune_config"]

## API Keys

In [6]:
HF_TOKEN = os.getenv("HUGGINGFACE_API_KEY")
if HF_TOKEN is None:
  raise EnvironmentError("A hugggingface token is necessary!")

## Helper Functions

In [10]:
def convert_to_messages_format(example):
    return [
        {"role": "user", "content": example['question']},
        {"role": "assistant", "content": example['answer']},
    ]

## Finetuning Function

In [11]:
def finetune(pipeline_config_name, finetune_config):
  max_seq_length = finetune_config["max_seq_length"] # Choose any! We auto support RoPE Scaling internally!
  dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
  load_in_4bit = finetune_config["load_in_4bit"] # Use 4bit quantization to reduce memory usage. Can be False.
  model, tokenizer = FastLanguageModel.from_pretrained(
      model_name = finetune_config["base_model_name"], # or choose "unsloth/Llama-3.2-1B-Instruct"
      # model_name = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
      max_seq_length = max_seq_length,
      dtype = dtype,
      load_in_4bit = load_in_4bit,
      # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
  )
  model = FastLanguageModel.get_peft_model(
      model,
      r = finetune_config["r"], # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
      target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                        "gate_proj", "up_proj", "down_proj",],
      lora_alpha = 16,
      lora_dropout = 0, # Supports any, but = 0 is optimized
      bias = "none",    # Supports any, but = "none" is optimized
      # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
      use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
      random_state = 3407,
      use_rslora = True,  # We support rank stabilized LoRA
      loftq_config = None, # And LoftQ
  )
  tokenizer = get_chat_template(
      tokenizer,
      chat_template = "llama-3.1",
  )
  dataset_finetune = load_dataset(
      FINETUNING_DATASET_NAME,
      pipeline_config_name,
      split="train",
      token=HF_TOKEN
  )
  print(dataset_finetune)
  print(dataset_finetune['question'][0])
  print(dataset_finetune['answer'][0])
  dataset_finetune = dataset_finetune.map(
      lambda x: {
          'conversations' : convert_to_messages_format(x)
          }
  )
  def formatting_prompts_func(examples):
      convos = examples["conversations"]
      texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
      return { "text" : texts, }
  dataset_finetune = dataset_finetune.map(formatting_prompts_func, batched = True)
  print(dataset_finetune['text'][0])
  trainer = SFTTrainer(
      model = model,
      tokenizer = tokenizer,
      train_dataset = dataset_finetune,
      dataset_text_field = "text",
      max_seq_length = max_seq_length,
      data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
      dataset_num_proc = 1,  # Affects memory usage
      packing = finetune_config["packing"], # Can make training 5x faster for short sequences.
      args = TrainingArguments(
          per_device_train_batch_size = finetune_config["batch_size"], # Affects memory usage
          gradient_accumulation_steps = finetune_config["gradient_accumulation_steps"],
          warmup_steps = 5,
          num_train_epochs = finetune_config["num_train_epochs"], # Set this for 1 full training run.
          # max_steps = 60,
          learning_rate = 2e-4,
          fp16 = not is_bfloat16_supported(),
          bf16 = is_bfloat16_supported(),
          logging_steps = 1,
          optim = "adamw_8bit",
          weight_decay = 0.01,
          lr_scheduler_type = "linear",
          seed = 3407,
          output_dir = "outputs",
          report_to = "none"
      ),
  )
  trainer = train_on_responses_only(
      trainer,
      instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
      response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
  )
  print(tokenizer.decode(trainer.train_dataset[0]["input_ids"]))
  space = tokenizer(" ", add_special_tokens = False).input_ids[0]
  print(tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]]))
  gpu_stats = torch.cuda.get_device_properties(0)
  start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
  max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
  print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
  print(f"{start_gpu_memory} GB of memory reserved.")
  trainer_stats = trainer.train()
  used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
  used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
  used_percentage = round(used_memory         /max_memory*100, 3)
  lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
  print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
  print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
  print(f"Peak reserved memory = {used_memory} GB.")
  print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
  print(f"Peak reserved memory % of max memory = {used_percentage} %.")
  print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")
  output_model_name = finetune_config["output_model_name"]
  model.push_to_hub_gguf(
          f"CPSC532/{output_model_name}",
          tokenizer,
          quantization_method = finetune_config["quantization_methods"],
          token = HF_TOKEN
      )

## Run Finetuning

In [ ]:
for include_source in config["matrix"]["pipeline_config.sources"]:
    for llm_config in config["matrix"]["pipeline_config.llm"]:
        pipeline_config_name = (
            f"src_{include_source}_llm_{llm_config['model']}".replace(
                "/", "-"
            )
        )
        for r in config["matrix"]["finetune_config.r"]:
          output_model_name = f"{pipeline_config_name}_r{r}"
          finetune_config = {
              **finetune_config_template,
              "r" : r,
              "output_model_name" : output_model_name
          }
          finetune(pipeline_config_name, finetune_config)


==((====))==  Unsloth 2024.11.10: Fast Llama patching. Transformers:4.46.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Dataset({
    features: ['filename', 'source', 'source_type', 'chunk', 'question', 'answer', 'pass_through'],
    num_rows: 750
})
What is the scoring calculation method used in the EQBench benchmark, as mentioned in 'AgentInstruct Toward Generative Teaching With Agentic Flows'?
According to the provided text, the scoring calculation method used in the EQBench benchmark is as follows:

"The metric scores are generated using both the version 1 and 2 implementations described in the EQBench paper and the creators’ github reposito

Map:   0%|          | 0/750 [00:00<?, ? examples/s]

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the scoring calculation method used in the EQBench benchmark, as mentioned in 'AgentInstruct Toward Generative Teaching With Agentic Flows'?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

According to the provided text, the scoring calculation method used in the EQBench benchmark is as follows:

"The metric scores are generated using both the version 1 and 2 implementations described in the EQBench paper and the creators’ github repository. The scoring calculation is calibrated such that a score of 0 corresponds to answering randomly, and a 100 would denote perfect alignment with the reference answer."

This indicates that the EQBench benchmark uses a scoring system that ranges from 0 to 100, where 0 represents random or incorrect answers and 100 represents perfect alignment with the reference a

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/19 [00:00<?, ? examples/s]

...

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Dataset({
    features: ['filename', 'source', 'source_type', 'chunk', 'question', 'answer', 'pass_through'],
    num_rows: 923
})
What are the 18 types of text modifications mentioned in the paper 'AgentInstruct Toward Generative Teaching With Agentic Flows'?
The paper does not mention "AgentInstruct Toward Generative Teaching With Agentic Flows" as the source of the text modifications. However, the text provided lists 18 types of text modifications:

1. Text Compression
2. Text Simplification
3. Text Expansion
4. Text Translation
5. Text Formatting
6. Sentiment Modification
7. Text Annotation
8. Keyword Replacement
9. Text Removing
10. Text Capitalization
11. Text Styling
12. Content Rewriting
13. Data Normalization
14. Plagiarism Rewording
15. Code Switching
16. Text Obfuscation
17. Textual Entailment
18. Rewriting with vocabulary limitations

These types of text modifications are mentioned in the provided text, but not in the paper "AgentInstruct Toward Generative Teaching With Age

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 28 | Num Epochs = 10
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 1
\        /    Total batch size = 1 | Total steps = 280
 "-____-"     Number of trainable parameters = 194,510,848


Step,Training Loss
1,1.605900
2,4.868800
3,0.739600
4,1.351000
5,1.158400
6,0.882900
7,0.882900
8,0.844200
9,0.846600
10,0.845500


In [ ]:
# import pandas as pd
# import matplotlib.pyplot as plt
# training_df = pd.DataFrame(trainer.state.log_history)

In [ ]:
# training_df.head(2)

In [ ]:
import sys
if not 'google.colab' in sys.modules:
  from helpers import create_training_plots
  fig = create_training_plots(training_df)
  fig.show()
  training_df.to_csv(f"training_logs/{OUTPUT_MODEL_NAME}.csv", index = False)

## Run Inference

In [ ]:
# from unsloth.chat_templates import get_chat_template

# tokenizer = get_chat_template(
#     tokenizer,
#     chat_template = "llama-3.1",
# )
# FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# def get_response(user_query):
#     messages = [
#     {"role": "user", "content": user_query},
#     ]
#     inputs = tokenizer.apply_chat_template(
#         messages,
#         tokenize = True,
#         add_generation_prompt = True, # Must add for generation
#         return_tensors = "pt",
#     ).to("cuda")

#     outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
#                             temperature = 1.5, min_p = 0.1)
#     return tokenizer.batch_decode(outputs)

In [ ]:
# dataset_finetune['question'][0]

Need to investigate how changing the question affects responses

In [ ]:
# resp = get_response(dataset_finetune['question'][0])
# print(resp[0].split("<|start_header_id|>assistant<|end_header_id|>")[1])